In [ ]:
BASE_PATH = r"C:\Users\sbanerje111521\OneDrive - GROUP DIGITAL WORKPLACE\Documents\Soumya_docs\python\poc_final"

In [ ]:
INPUT_FULL_PATH = BASE_PATH + r"\inputs\BusinessDataDictionnary.xlsx"
OUTPUT_PATH = BASE_PATH + r"\outputs"
MODEL_PATH = "C:\\Users\\sbanerje111521\\OneDrive - GROUP DIGITAL WORKPLACE\\Documents\\Soumya_docs\\python\\all-MiniLM-L6-v2"

In [ ]:
# read data
import pickle
with open(BASE_PATH +r"\lemmatized_result.pkl", 'rb') as rfile:
    df = pickle.load(rfile)
df.shape

In [ ]:
df.columns

# KMeans

In [ ]:
from sentence_transformers import SentenceTransformer

from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

# model
embedder = SentenceTransformer(MODEL_PATH)

In [ ]:
# corpus = df['clean_def'][:50].values
# corpus_df = df['clean_def'][:50]
corpus_df = df[['clean_def','Term','Definition','Identifiant technique','Global ID (SG)']]
corpus_df = pd.DataFrame(corpus_df)
corpus_df.reset_index(drop=True, inplace=True)
corpus_df.head(2)

In [ ]:
corpus_embeddings = embedder.encode(corpus_df['clean_def'].values)
print("embedding done")

# normalize
corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)
print("normalize done")

In [ ]:
# save the embeddings
import pickle
with open(OUTPUT_PATH + r'\embeddings_clean_def.pkl', 'wb') as wfile:
    pickle.dump(corpus_embeddings, wfile)

In [ ]:
clustering_model = KMeans(n_clusters=5, random_state=0)
clustering_model.fit(corpus_embeddings)
print("fit done")
cluster_assignment = clustering_model.labels_

In [ ]:
# save the embeddings
import pickle
with open(OUTPUT_PATH + r'\kmeans_baseline.pkl', 'wb') as wfile:
    pickle.dump(clustering_model, wfile)

In [ ]:
import pickle
with open(OUTPUT_PATH + r'\kmeans_baseline.pkl', 'rb') as rfile:
    clustering_model = pickle.load(rfile)

In [ ]:
clustering_model.labels_

In [ ]:
corpus_df['label'] = cluster_assignment
corpus_df

In [ ]:
# save to excel file
from utilities import current_time
VERSION = current_time()
corpus_df.to_excel(OUTPUT_PATH+'\\'+f'kear_kmeans_{VERSION}.xlsx', index=False)

In [ ]:

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

#print(clustered_sentences)
for k,v in clustered_sentences.items():
    print(k)
    for val in v:
        print(val)
    print("\n\n")

## Wordcloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
corpus_df.head(3)

In [ ]:
# text.to_string()
' '.join(text)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
text = corpus_df[corpus_df['label']==4].iloc[:,0].values
wordcloud = WordCloud().generate(' '.join(text))
plt.imshow(wordcloud)